In [1]:
from functools import partial
from operator import itemgetter

from langchain_core.prompts import PromptTemplate, format_document
from langchain_core.output_parsers import  StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 加载arXiv上的论文ReAct: Synergizing Reasoning and Acting in Language Models
loader = ArxivLoader(query = "2210.03629", load_max_docs = 1)
docs = loader.load()

# 把文本分割成500个字符为一组的片段
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)
chunks = text_splitter.split_documents(docs)

llm = ChatOllama(model = 'qwen2.5')

# 构建工具函数：将Document转换成字符串
document_prompt = PromptTemplate.from_template('{page_content}')
partial_format_document = partial(format_document, prompt = document_prompt)

# 构建Context链：总结第一个文档并作为后续总结的上下文
first_prompt = PromptTemplate.from_template('Summarize this content(用中文): \n\n {context}')
context_chain = (
        {'context': partial_format_document}
        | first_prompt
        | llm
        | StrOutputParser()
)

# 构建Refine链：基于上下文（上一次的总结）和当前内容进一步总结
refine_prompt = PromptTemplate.from_template(
    "Here's your first summary: {prev_response}."
    "Now add to it based on the following context: {context}"
)
refine_chain = (
    {
        "prev_response": itemgetter("prev_response"),
        "context": lambda x: partial_format_document(x['doc']),
    }
    | refine_prompt
    | llm
    | StrOutputParser()
)

# 构建一个负责执行Refine循环的函数
def refine_loop(docs):
    summary = context_chain.invoke(docs[0])
    for i, doc in enumerate(docs[1:]):
        summary = refine_chain.invoke({'prev_response': summary, 'doc': doc})
        return summary

refine_loop(chunks[:4])

'该内容摘要如下：\n\n发表于2023年ICLR会议的论文《REACT：语言模型中推理与行动的协同》介绍了在大规模语言模型（LLM）中结合推理和行动的方法。该研究由普林斯顿大学计算机科学系及Google Research Brain团队共同完成，作者包括Shunyu Yao、Jeffrey Zhao等。\n\n摘要指出，在语言理解和交互式决策制定的各种任务中，语言模型在推理（例如：链式思维提示）与执行（例如：生成行动计划）方面的能力通常被分别研究。本文提出了一种新的方法REACT（Reasoning and Acting in Language Models），旨在以交错的方式让LLM生成推理痕迹和特定任务的行动，从而在两者之间实现更高的协同效应：推理痕迹帮助模型在生成、跟踪和更新行动计划以及处理复杂情境时更具灵活性。\n\n这种方法不仅增强了模型解决复杂问题的能力，还提高了其在实际应用中的实用性和灵活性。通过这种方式，REACT为构建更加智能且能够有效执行任务的语言模型提供了新的思路和方法。'